In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import re
from bs4 import BeautifulSoup # For HTML parsing
import requests # Website connections
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
import pandas as pd # For converting results to a dataframe and bar chart plots
import json # For parsing json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from time import sleep
from tqdm.notebook import tqdm

In [2]:
def savetocsv(dfname,filename):
    """
    Save Pandas DF to csv file 
    """
    dfname.to_csv(r'/Users/idmp/GoogleDrive/melbourne-housing-market/{}.csv'.format(filename), index = False)

In [124]:
driver = webdriver.Chrome('/Users/idmp/GoogleDrive/chromedriver')

In [4]:
df=pd.read_csv('/Users/idmp/GoogleDrive/melbourne-housing-market/Melbourne_housing_FULL.csv', engine='python')  
pcodes=pd.DataFrame(df['Postcode'].dropna().unique())   
plist=[str(pcodes.iloc[i,0])[:-2] for i in range(pcodes.shape[0])]

In [119]:
data=[]
counter=0
for code in tqdm(plist):
    url2=f'https://quickstats.censusdata.abs.gov.au/census_services/getproduct/census/2016/quickstat/POA{code}?opendocument'
    driver.get(url2)
    sleep(3)
    pop = driver.find_element_by_xpath('/html/body/div[3]/div[4]/div/div[1]/div[1]/div[2]/table[1]/tbody/tr[1]/td').text.replace(',','')
    households = driver.find_element_by_xpath('/html/body/div[3]/div[4]/div/div[1]/div[1]/div[2]/table[2]/tbody/tr[1]/td').text.replace(',','')
    medage = driver.find_element_by_xpath('/html/body/div[3]/div[4]/div/div[1]/div[1]/div[2]/table[1]/tbody/tr[4]/td').text
    wkinc = driver.find_element_by_xpath('/html/body/div[3]/div[4]/div/div[1]/div[1]/div[2]/table[3]/tbody/tr[3]/td').text.replace(',','').replace('$','')
    mtg = driver.find_element_by_xpath('/html/body/div[3]/div[4]/div/div[1]/div[1]/div[2]/table[3]/tbody/tr[4]/td').text.replace(',','').replace('$','')
    rent = driver.find_element_by_xpath('/html/body/div[3]/div[4]/div/div[1]/div[1]/div[2]/table[3]/tbody/tr[5]/td').text.replace(',','').replace('$','')
    cars = driver.find_element_by_xpath('/html/body/div[3]/div[4]/div/div[1]/div[1]/div[2]/table[3]/tbody/tr[6]/td').text
    univ = driver.find_element_by_xpath('/html/body/div[3]/div[4]/div/div[3]/div[1]/table[5]/tbody/tr[10]/td[1]').text.replace(',','')
    unemp = driver.find_element_by_xpath('/html/body/div[3]/div[4]/div/div[3]/div[1]/table[14]/tbody/tr[5]/td[1]').text.replace(',','')
    data.append([pop,households,medage,wkinc,mtg, rent, cars,univ,unemp])

In [232]:
columns_names =['population',
                'households',
                'med_age',
                'hh_inc',
                'hh_mtg_pmt',
                'med_wk_rent',
                'hh_cars',
                'uni_edu',
                'unemp']
dataframe = pd.DataFrame(data, columns=columns_names, index=plist).astype('float')
dataframe=dataframe.reset_index()

In [236]:
savetocsv(dataframe,'popstats')